<a href="https://colab.research.google.com/github/nihemelandu/churn_clv_prediction/blob/main/02a_Initial_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd

In [12]:
import warnings
warnings.filterwarnings('ignore')

# Configuration - easy to modify in notebook
SAMPLE_RATE = 0.005
RANDOM_SEED = 42
MIN_SAMPLE_SIZE = 10000
MAX_REASONABLE_PRICE = 10000
EXPECTED_DATE_START = '2019-10-01'
EXPECTED_DATE_END = '2019-12-01'

print("✓ Libraries imported and configuration set")

✓ Libraries imported and configuration set


Data Loading & Sampling

1.  Load data with appropriate sampling strategy
2.  Set random seed for reproducibility
3.  Document sample size and sampling rationale
4. Verify sample loaded successfully

In [13]:
# Data Loading with Reproducible Sampling

print("=== LOADING DATA WITH REPRODUCIBLE SAMPLING ===")

# Set seed for reproducibility
np.random.seed(RANDOM_SEED)

# Load samples from both files
files = ['/content/drive/MyDrive/2019-Oct.csv', '/content/drive/MyDrive/2019-Nov.csv']
samples = []

for file in files:
    print(f"\nLoading {file}...")
    try:
        sample = pd.read_csv(file, skiprows=lambda i: i > 0 and np.random.random() > SAMPLE_RATE)
        print(f"✓ Loaded {len(sample):,} rows from {file}")
        samples.append(sample)
    except FileNotFoundError:
        print(f"✗ {file} not found")
    except Exception as e:
        print(f"✗ Error loading {file}: {e}")

# Combine samples
if samples:
    sample_df = pd.concat(samples, ignore_index=True)
    print(f"\n✓ Combined sample: {len(sample_df):,} rows, {sample_df.shape[1]} columns")

    if len(sample_df) < MIN_SAMPLE_SIZE:
        print(f"⚠️  Warning: Sample size below minimum {MIN_SAMPLE_SIZE:,}")
else:
    raise ValueError("No data could be loaded!")

=== LOADING DATA WITH REPRODUCIBLE SAMPLING ===

Loading /content/drive/MyDrive/2019-Oct.csv...
✓ Loaded 212,521 rows from /content/drive/MyDrive/2019-Oct.csv

Loading /content/drive/MyDrive/2019-Nov.csv...
✓ Loaded 337,154 rows from /content/drive/MyDrive/2019-Nov.csv

✓ Combined sample: 549,675 rows, 9 columns


Schema Validation

*   Data type consistency checks
*   Column name validation
*   Expected vs actual data types comparison
*   Data type optimization opportunities

In [14]:
# Schema Validation

print("=== SCHEMA VALIDATION ===")

# Expected columns for eCommerce events
expected_columns = ['event_time', 'event_type', 'product_id', 'category_id',
                   'category_code', 'brand', 'price', 'user_id', 'user_session']

print(f"Actual columns: {sample_df.columns.tolist()}")

# Check for missing/extra columns
missing_cols = [col for col in expected_columns if col not in sample_df.columns]
extra_cols = [col for col in sample_df.columns if col not in expected_columns]

if missing_cols:
    print(f"✗ Missing columns: {missing_cols}")
else:
    print("✓ All expected columns present")

if extra_cols:
    print(f"⚠️  Extra columns: {extra_cols}")

# Data types
print(f"\nData types:")
print(sample_df.dtypes)

=== SCHEMA VALIDATION ===
Actual columns: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']
✓ All expected columns present

Data types:
event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object


In [17]:
#Data Type Validation

print("=== DATA TYPE VALIDATION ===")

# Validate numeric columns
numeric_cols = ['product_id', 'category_id', 'price', 'user_id']
for col in numeric_cols:
    if col in sample_df.columns:
        try:
            pd.to_numeric(sample_df[col], errors='raise')
            print(f"✓ {col} is properly numeric")
        except ValueError:
            print(f"✗ {col} has non-numeric values")

# Validate datetime
if 'event_time' in sample_df.columns:
    try:
        sample_df['event_time_parsed'] = pd.to_datetime(sample_df['event_time'], utc=True)

        # Check date range
        min_date = sample_df['event_time_parsed'].min()
        max_date = sample_df['event_time_parsed'].max()

        print(f"✓ event_time parsed successfully")
        print(f"Date range: {min_date.date()} to {max_date.date()}")

        # Validate expected date range
        expected_start = pd.to_datetime(EXPECTED_DATE_START, utc=True)
        expected_end = pd.to_datetime(EXPECTED_DATE_END, utc=True)

        if min_date < expected_start or max_date > expected_end:
            print(f"⚠️  Dates outside expected range {EXPECTED_DATE_START} to {EXPECTED_DATE_END}")
        else:
            print("✓ Dates within expected range")

    except Exception as e:
        print(f"✗ event_time validation failed: {e}")

=== DATA TYPE VALIDATION ===
✓ product_id is properly numeric
✓ category_id is properly numeric
✓ price is properly numeric
✓ user_id is properly numeric
✓ event_time parsed successfully
Date range: 2019-10-01 to 2019-11-30
✓ Dates within expected range


In [18]:
# Data Quality Analysis

print("=== DATA QUALITY ANALYSIS ===")

# Missing values
print("Missing values by column:")
missing_summary = sample_df.isnull().sum()
missing_pct = (missing_summary / len(sample_df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_pct.round(2)
}).sort_values('Missing_Count', ascending=False)

# Show only columns with missing values
missing_issues = missing_df[missing_df['Missing_Count'] > 0]
if len(missing_issues) > 0:
    display(missing_issues)
else:
    print("✓ No missing values found")

=== DATA QUALITY ANALYSIS ===
Missing values by column:


,Missing_Count,Missing_Percentage
category_code,177156,32.23
brand,76458,13.91


In [19]:
# Uniqueness Analysis

print("=== UNIQUENESS ANALYSIS ===")

uniqueness_stats = []
for col in sample_df.columns:
    unique_count = sample_df[col].nunique()
    unique_pct = (unique_count / len(sample_df)) * 100
    uniqueness_stats.append({
        'Column': col,
        'Unique_Values': unique_count,
        'Uniqueness_Pct': round(unique_pct, 2),
        'Data_Type': str(sample_df[col].dtype)
    })

uniqueness_df = pd.DataFrame(uniqueness_stats)
display(uniqueness_df)

=== UNIQUENESS ANALYSIS ===


,Column,Unique_Values,Uniqueness_Pct,Data_Type
0,event_time,504069,91.70,object
1,event_type,3,0.00,object
2,product_id,74253,13.51,int64
3,category_id,650,0.12,int64
4,category_code,128,0.02,object
5,brand,3039,0.55,object
6,price,35669,6.49,float64
7,user_id,426829,77.65,int64
8,user_session,530289,96.47,object
9,event_time_parsed,504069,91.70,"datetime64[ns, UTC]"


In [20]:
# Business Logic Validation

print("=== BUSINESS LOGIC VALIDATION ===")

# Event types validation
if 'event_type' in sample_df.columns:
    print("Event types found:")
    event_counts = sample_df['event_type'].value_counts()
    display(event_counts)

    expected_events = ['view', 'cart', 'purchase', 'remove_from_cart']
    actual_events = set(sample_df['event_type'].unique())
    unexpected = actual_events - set(expected_events)

    if unexpected:
        print(f"⚠️  Unexpected event types: {unexpected}")
    else:
        print("✓ All event types are expected")

# Price validation
if 'price' in sample_df.columns:
    print(f"\nPrice validation:")
    print(sample_df['price'].describe())

    zero_prices = (sample_df['price'] == 0).sum()
    negative_prices = (sample_df['price'] < 0).sum()
    high_prices = (sample_df['price'] > MAX_REASONABLE_PRICE).sum()

    print(f"\nPrice issues:")
    print(f"Zero prices: {zero_prices:,}")
    print(f"Negative prices: {negative_prices:,}")
    print(f"High prices (>${MAX_REASONABLE_PRICE:,}): {high_prices:,}")

=== BUSINESS LOGIC VALIDATION ===
Event types found:


,count
event_type,
view,522020
cart,19381
purchase,8274


✓ All event types are expected

Price validation:
count    549675.000000
mean        292.024049
std         358.140139
min           0.000000
25%          67.960000
50%         164.980000
75%         360.110000
max        2574.070000
Name: price, dtype: float64

Price issues:
Zero prices: 1,224
Negative prices: 0
High prices (>$10,000): 0


In [22]:
# Cell 8: Memory Usage Analysis

print("=== MEMORY USAGE ANALYSIS ===")

# Current sample memory
memory_usage = sample_df.memory_usage(deep=True)
total_memory_mb = memory_usage.sum() / (1024 * 1024)

print(f"Sample size: {len(sample_df):,} rows")
print(f"Sample memory: {total_memory_mb:.1f} MB")

print(f"\nMemory by column:")
for col, usage in memory_usage.items():
    usage_mb = usage / (1024 * 1024)
    print(f"{col}: {usage_mb:.1f} MB")

# Estimate full dataset
scale_factor = 285_000_000 / len(sample_df)
estimated_gb = (total_memory_mb * scale_factor) / 1024
print(f"\nEstimated full dataset: {estimated_gb:.1f} GB")

=== MEMORY USAGE ANALYSIS ===
Sample size: 549,675 rows
Sample memory: 189.0 MB

Memory by column:
Index: 0.0 MB
event_time: 37.7 MB
event_type: 27.8 MB
product_id: 4.2 MB
category_id: 4.2 MB
category_code: 30.8 MB
brand: 27.1 MB
price: 4.2 MB
user_id: 4.2 MB
user_session: 44.6 MB
event_time_parsed: 4.2 MB

Estimated full dataset: 95.7 GB


In [23]:
# Summary and Next Steps

print("=== EXPLORATION SUMMARY ===")
print("✓ Reproducible sampling completed")
print("✓ Schema validation performed")
print("✓ Data quality issues identified")
print("✓ Memory usage analyzed")

print(f"\nKey findings:")
print(f"- Sample size: {len(sample_df):,} rows")
print(f"- Date range: {sample_df['event_time_parsed'].min().date()} to {sample_df['event_time_parsed'].max().date()}")
print(f"- Unique users: {sample_df['user_id'].nunique():,}")
print(f"- Event types: {sample_df['event_type'].nunique()}")

print(f"\nReady for Notebook 02b - Data Cleaning")

=== EXPLORATION SUMMARY ===
✓ Reproducible sampling completed
✓ Schema validation performed
✓ Data quality issues identified
✓ Memory usage analyzed

Key findings:
- Sample size: 549,675 rows
- Date range: 2019-10-01 to 2019-11-30
- Unique users: 426,829
- Event types: 3

Ready for Notebook 02b - Data Cleaning


Your data tells you what it wants to be:
→ Comparing categories? Bar charts work great
→ Showing change over time? Line charts are your friend
→ Looking for relationships? Use Scatter plots
→ Breaking down a whole? Pie charts (sparingly) or stacked bars
→ Showing distribution? Histograms tell the real story

I now spend 5 minutes before every analysis asking myself:
— Am I comparing?
— Am I tracking change?
— Am I looking for patterns?
— Am I showing composition?